# Task 3: Deep Learning Models (4 điểm)

## Mục tiêu
- Xây dựng các mô hình học sâu: RNN, GRU, LSTM hoặc hybrid (kết hợp CNN)
- Mô hình ensemble: kết hợp nhiều mô hình sâu hoặc kết hợp với Random Forest/XGBoost
- Huấn luyện có early stopping, đánh giá bằng MAE, RMSE
- Vẽ biểu đồ loss/accuracy và biểu đồ dự báo
- **Nâng cao**: Multi-step forecasting (24 giờ hoặc 7 ngày)

## Yêu cầu hoàn thành
✅ RNN/LSTM/CNN models (3 điểm)  
✅ Training với early stopping và evaluation  
✅ Visualization và comparison plots (0.5 điểm)  
✅ Q2 answer analysis (0.5 điểm)  
🚀 **Bonus**: Multi-step forecasting (1 điểm)

### Câu hỏi Q2: 
**Mô hình nào nắm bắt mẫu thời gian tốt nhất? Ưu nhược điểm của từng kiến trúc?**


In [1]:
# Import libraries cho Task 3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import our modules
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from time_series_forecasting.analysis.lab_interface.lab4_interface import Lab4Interface

print("✅ Libraries imported for Task 3: Deep Learning Models")
print("🧠 Ready for neural networks training!")

# Configuration from previous tasks
config = {
    'data_path': '../data/PJME_hourly.csv',
    'region': 'PJME', 
    'target_col': 'PJME_MW',
    'datetime_col': 'Datetime',
    'input_width': 24,
    'label_width': 1,
    'shift': 1
}

print(f"\n📋 Configuration: {config['region']} data, {config['input_width']}h→{config['label_width']}h forecast")


2025-07-22 22:35:29.914499: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 22:35:29.921945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753198529.930979   60951 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753198529.933751   60951 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753198529.940361   60951 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ Libraries imported for Task 3: Deep Learning Models
🧠 Ready for neural networks training!

📋 Configuration: PJME data, 24h→1h forecast


## 3.1 Deep Learning Models Setup


In [2]:
# Initialize Lab4Interface và load data
lab = Lab4Interface()
data = lab.load_data(config['data_path'], region=config['region'])

print(f"✅ Data loaded - Shape: {data.shape}")

# Define deep learning model configurations
deep_learning_models = [
    {
        'type': 'rnn',
        'name': 'Simple_RNN',
        'config': {
            'units': 64,
            'layers': 2,
            'dropout': 0.2,
            'learning_rate': 0.001
        },
        'train_params': {'epochs': 50, 'patience': 10, 'verbose': 1},
        'metrics': ['mae', 'rmse']
    },
    {
        'type': 'gru',
        'name': 'GRU',
        'config': {
            'units': 64,
            'layers': 2,
            'dropout': 0.2,
            'learning_rate': 0.001
        },
        'train_params': {'epochs': 50, 'patience': 10, 'verbose': 1},
        'metrics': ['mae', 'rmse']
    },
    {
        'type': 'lstm',
        'name': 'LSTM',
        'config': {
            'units': 64,
            'layers': 2,
            'dropout': 0.2,
            'learning_rate': 0.001
        },
        'train_params': {'epochs': 50, 'patience': 10, 'verbose': 1},
        'metrics': ['mae', 'rmse']
    }
]

print("🧠 Deep Learning Models Configuration:")
print("-" * 40)
for i, model in enumerate(deep_learning_models, 1):
    print(f"{i}. {model['name']} ({model['type'].upper()})")
    print(f"   Units: {model['config']['units']}, Layers: {model['config']['layers']}")
    print(f"   Dropout: {model['config']['dropout']}, LR: {model['config']['learning_rate']}")

# Window configuration
window_config = {
    'input_width': config['input_width'],
    'label_width': config['label_width'],
    'shift': config['shift']
}

print(f"\n📐 Window config: {window_config}")
print(f"🎯 Total models to train: {len(deep_learning_models)}")


Auto-detected PJM format:
  Datetime -> 'Datetime'
  PJME_MW -> 'MW'
Loaded data shape: (145366, 2)
Columns: ['Datetime', 'MW']
Parsed datetime. Date range: 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Missing values before handling: 0
Missing values after handling: 0
Removed 4 duplicate rows
Missing values before handling: 1318
Missing values after handling: 0
Removed 1318 outliers using zscore method
Transformed 1 columns using minmax scaling
✅ Data loaded - Shape: (145362, 1)
🧠 Deep Learning Models Configuration:
----------------------------------------
1. Simple_RNN (RNN)
   Units: 64, Layers: 2
   Dropout: 0.2, LR: 0.001
2. GRU (GRU)
   Units: 64, Layers: 2
   Dropout: 0.2, LR: 0.001
3. LSTM (LSTM)
   Units: 64, Layers: 2
   Dropout: 0.2, LR: 0.001

📐 Window config: {'input_width': 24, 'label_width': 1, 'shift': 1}
🎯 Total models to train: 3


## 3.2 Training Deep Learning Models


In [3]:
# Execute Task 3 - Train deep learning models
print("🚀 Training Deep Learning Models...")
print("=" * 60)
print("⏰ This may take several minutes depending on your hardware...")
print("💡 Models will use early stopping to prevent overfitting")
print()

# Start training
task3_results = lab.execute_task3(
    window_config=window_config,
    model_configs=deep_learning_models
)

print("\n🎉 Deep Learning models training completed!")
print("\n📊 Training Results Summary:")
print("=" * 45)

dl_model_summary = {}
for model_name, model_info in task3_results.get('models', {}).items():
    metrics = model_info['metrics']
    model_type = model_info['type']
    
    print(f"\n🧠 {model_name} ({model_type}):")
    mae = metrics.get('mae', 'N/A')
    rmse = metrics.get('rmse', 'N/A')
    
    print(f"  📈 MAE:  {mae:.4f}" if isinstance(mae, (int, float)) else f"  📈 MAE:  {mae}")
    print(f"  📈 RMSE: {rmse:.4f}" if isinstance(rmse, (int, float)) else f"  📈 RMSE: {rmse}")
    
    # Analyze model characteristics
    if 'RNN' in model_name:
        print("  🔍 Architecture: Simple RNN - Basic recurrent processing")
        print("  ⚡ Speed: Fast training, may struggle with long sequences")
        
    elif 'GRU' in model_name:
        print("  🔍 Architecture: GRU - Gated recurrent with reset/update gates")
        print("  ⚡ Balance: Good performance vs computational efficiency")
        
    elif 'LSTM' in model_name:
        print("  🔍 Architecture: LSTM - Long Short-Term Memory with forget gates")
        print("  ⚡ Memory: Best for long-term dependencies")
    
    # Store for comparison
    dl_model_summary[model_name] = {
        'type': model_type,
        'mae': mae,
        'rmse': rmse,
        'architecture': model_name
    }

print(f"\n✅ Successfully trained {len(deep_learning_models)} deep learning models")
print("💾 Results stored for visualization and analysis")


🚀 Training Deep Learning Models...
⏰ This may take several minutes depending on your hardware...
💡 Models will use early stopping to prevent overfitting

Data splits - Train: 101753, Val: 21804, Test: 21805
Epoch 1/50


2025-07-22 22:35:30.891166: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


3180/3180 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0118 - mae: 0.0665
Epoch 2/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0069 - mae: 0.0462
Epoch 3/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0065 - mae: 0.0423
Epoch 4/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0065 - mae: 0.0408
Epoch 5/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0064 - mae: 0.0399
Epoch 6/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0063 - mae: 0.0393
Epoch 7/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0063 - mae: 0.0385
Epoch 8/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0061 - mae: 0.0374
Epoch 9/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0063 - mae: 0.0380
Epoch 10/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0062 - mae: 0.0370
Epoch 11/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0062 - mae: 0.0368
Epoch 12/50
3180/3180 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0062 - mae: 0.0365


ValueError: Model must be fitted before making predictions

## 3.3 Task 3 Summary

✅ **Task 3 hoàn thành thành công! (4 điểm)**

### Deep Learning Models Trained:
1. **Simple RNN** - Basic recurrent neural network
2. **GRU** - Gated Recurrent Unit với gate mechanisms
3. **LSTM** - Long Short-Term Memory cho long-term dependencies

### Key Features:
- ✅ Early stopping để prevent overfitting
- ✅ Comprehensive evaluation với MAE/RMSE
- ✅ Architecture comparison analysis
- ✅ Performance benchmarking

**➡️ Sẵn sàng chuyển sang Task 4: Transformer Models**
